# Registar um pipeline no mlflow

Um pipeline não é mais que a composição de vários modelos/transformações

In [3]:
import mlflow

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [5]:
ROOT_PATH = '../../data/'
SEED = 3
TARGET_COL = "Outcome"

## Definir a diretoria onde as experiências são guardadas

In [ ]:
from pathlib import Path

uri = "http://localhost:5000"

mlflow.set_tracking_uri(uri)

## Fazer set da experiência "Diabetes Prediction Experiment"

In [7]:
mlflow.set_experiment("Diabetes Prediction Experiment")

MlflowException: API request to http://0.0.0.0:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='0.0.0.0', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Diabetes+Prediction+Experiment (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002335FDB1520>: Failed to establish a new connection: [WinError 10049] O endereço pedido não é válido no seu contexto'))

## Criar os datasets

In [ ]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

X_train.head()

## Criar uma run

In [ ]:
run = mlflow.start_run(run_name="Linear Regression Run - C0.1 - pipeline")
RUN_ID = run.info.run_uuid
RUN_ID

## Guardar datasets, modelos, artefactos, métricas e parametros da run

In [ ]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name="Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name="Diabetes Test Dataset")
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

# Guardamos a seed utilizado como parametro
mlflow.log_param("seed", SEED)

In [ ]:
lr_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("logistic_regression", LogisticRegression(random_state=SEED, C=0.1))
])
lr_pipeline.fit(X_train, y_train)
mlflow.sklearn.log_model(lr_pipeline, artifact_path="lr_pipeline", registered_model_name="logistic_reg")
lr_pipeline

In [ ]:
params=lr_pipeline.get_params()

modified_params = {}
for k, v in params.items():
    new_key = k.replace("logistic_regression__", '')
    modified_params[new_key] = v

mlflow.log_params(modified_params)
modified_params

In [ ]:
y_preds = lr_pipeline.predict(X_test)
acc = accuracy_score(y_test, y_preds)
mlflow.log_metric("accuracy", acc)
acc

## Terminar a run

In [ ]:
mlflow.end_run()

## Consultar uma run já concluida

In [ ]:
run = mlflow.get_run(RUN_ID)

In [ ]:
run.data

## Ver a experiência na UI do mlflow

A UI do mlflow permite ver de forma visual todas as experiências criadas e permite por exemplo, comparar, filtar e ordenar, as runs dentro de uma experiência de forma visual.

Para correr a UI do mflow é necessário executar, na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri ./mlruns`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow. Caso tenham problemas, considerem instalar o `mlflow-ui`, ao invés do mlflow.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`http://127.0.0.1:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Experiments` podem explorar as experiências e runs que criaram.